In [1]:
import os                                                                       
import sys                                                                      
import struct                                                                   
import numpy as np                                                              
import matplotlib.pyplot as plt                                                 
from PIL import Image 
import numpy as np
from datetime import datetime
from scipy.stats import norm
from scipy.stats import multivariate_normal as mvn
from scipy.stats import binom

In [2]:
#HW2-1
f = open('train-images.idx3-ubyte','rb')
image_size = 28
num_images = 60000

f.read(16)
buf = f.read(image_size * image_size * num_images)
train_X = np.frombuffer(buf, dtype=np.uint8).astype(np.float32)
train_X = train_X.reshape(num_images, image_size, image_size, 1)
f = open('t10k-images.idx3-ubyte','rb')
image_size = 28
num_images = 10000

f.read(16)
buf = f.read(image_size * image_size * num_images)
test_X = np.frombuffer(buf, dtype=np.uint8).astype(np.float32)
test_X = test_X.reshape(num_images, image_size, image_size, 1)
f = open('train-labels.idx1-ubyte','rb')
num_label = 60000

f.read(8)
buf = f.read(num_label)
train_y = np.frombuffer(buf, dtype=np.uint8).astype(np.float32)
train_y = train_y.reshape(num_label, 1)
f = open('t10k-labels.idx1-ubyte','rb')
num_label = 10000

f.read(8)
buf = f.read(num_label)
test_y = np.frombuffer(buf, dtype=np.uint8).astype(np.float32)
test_y = test_y.reshape(num_label, 1)

In [3]:
class NaiveBayes(object):
    def fit(self, X, Y, smoothing=1.5e2):
        self.gaussians = dict()
        self.priors = dict()
        labels = set(train_y.reshape(1,-1)[0])
        Y=train_y.reshape(1,-1)[0]
        for c in labels:
            
            current_x = X[Y == c]
            self.gaussians[c] = {
                'mean': current_x.mean(axis=0),
                'var': current_x.var(axis=0) + smoothing,
            }
            
            self.priors[c] = float(len(Y[Y == c])) / len(Y)
            
        
    def score(self, X, Y):
        Y=Y.reshape(1,-1)[0]
        Predict,P ,Prob= self.predict(X)
        marginal = np.sum(P,axis=1)
        
        if X.shape[0]==10000:
            for index in range(X.shape[0]):
                print('Postirior(log):\n')
                for number in range(10):
                    print('%d:%f'%(number,P[index,number]/marginal[index]))
                print('Prediction:%d , Ans: %d'%(Predict[index],Y[index]))
            for cla in range(10):
                #print(Prob.shape,Prob[index].shape)
                #fig=np.zeros((28,28))
                #plt.subplots(figsize=(2,2))
                fig=Prob[cla][:,:,cla].reshape(28,28)-self.gaussians[cla]['mean'].reshape(28,28)
                max_f=np.max(fig)
                min_f=np.min(fig)
                process=np.zeros((28,28))
                result=np.zeros((28,28))
                for row in range(28):
                    for col in range(28):
                        process[row][col]=(fig[row][col]-min_f)/(max_f-min_f)*(255)
                        if process[row][col]<128:
                            result[row][col] = 1
                print(result)
                #plt.imshow(fig,cmap = plt.cm.gray)
        return np.mean(Predict != Y)

    def predict(self, X):
        
        N, D, _ ,_ = X.shape
        K = len(self.gaussians)
        P = np.zeros((N, K))
        Prob = np.zeros((N,D,D,K))
        for i in range(K):
            g=self.gaussians[i]
            mean, var = g['mean'], g['var']
            Prob[:,:,:,i]= 1 / np.sqrt(2 * np.pi * var.reshape(28,28)) * np.exp(-np.square(X.reshape(X.shape[0],28,28) - mean.reshape(28,28))/(2 * var.reshape(28,28)))
            P[:,i] = mvn.logpdf(X.reshape(X.shape[0], 784), mean=mean.reshape(784), cov=var.reshape(784)) + np.log(self.priors[i])
            
        return np.argmax(P, axis=1),P,Prob


if __name__ == '__main__':
    Xtrain, Ytrain = train_X, train_y
    Xtest, Ytest = test_X, test_y

    model = NaiveBayes()
    t0 = datetime.now()
    model.fit(Xtrain, Ytrain)
    #print("Training time:", (datetime.now() - t0))

    #t0 = datetime.now()
    #print("Train accuracy:", model.score(Xtrain, Ytrain))
    #print("Time to compute train accuracy:", (datetime.now() - t0), "Train size:", len(Ytrain))

    #t0 = datetime.now()
    
    print("Error rate:", model.score(Xtest, Ytest))
    #print("Time to compute test accuracy:", (datetime.now() - t0), "Test size:", len(Ytest))

Postirior(log):

0:0.104845
1:0.132238
2:0.108492
3:0.092595
4:0.090945
5:0.090967
6:0.124656
7:0.079860
8:0.092847
9:0.082555
Prediction:7 , Ans: 7
Postirior(log):

0:0.086198
1:0.119174
2:0.070609
3:0.083970
4:0.101334
5:0.085893
6:0.082219
7:0.150208
8:0.092300
9:0.128094
Prediction:2 , Ans: 2
Postirior(log):

0:0.107237
1:0.087125
2:0.100145
3:0.099698
4:0.099503
5:0.099238
6:0.099240
7:0.106963
8:0.098538
9:0.102313
Prediction:1 , Ans: 1
Postirior(log):

0:0.080572
1:0.203134
2:0.087268
3:0.087009
4:0.089141
5:0.085602
6:0.083932
7:0.103716
8:0.085456
9:0.094171
Prediction:0 , Ans: 0
Postirior(log):

0:0.094604
1:0.160594
2:0.095009
3:0.095685
4:0.089325
5:0.093033
6:0.096874
7:0.092723
8:0.092595
9:0.089559
Prediction:4 , Ans: 4
Postirior(log):

0:0.110977
1:0.087272
2:0.101000
3:0.100185
4:0.099775
5:0.099765
6:0.103816
7:0.100943
8:0.098424
9:0.097844
Prediction:1 , Ans: 1
Postirior(log):

0:0.097319
1:0.146924
2:0.097969
3:0.094754
4:0.089918
5:0.092147
6:0.102322
7:0.094124
8

3:0.088547
4:0.100135
5:0.091479
6:0.123702
7:0.100087
8:0.090206
9:0.096471
Prediction:3 , Ans: 3
Postirior(log):

0:0.099392
1:0.101578
2:0.100491
3:0.102826
4:0.087336
5:0.082641
6:0.094525
7:0.127045
8:0.086557
9:0.117609
Prediction:5 , Ans: 6
Postirior(log):

0:0.101168
1:0.109121
2:0.104641
3:0.095755
4:0.094216
5:0.092829
6:0.126883
7:0.090852
8:0.092794
9:0.091741
Prediction:7 , Ans: 5
Postirior(log):

0:0.087449
1:0.135542
2:0.092867
3:0.089502
4:0.097900
5:0.083759
6:0.108964
7:0.110671
8:0.085808
9:0.107538
Prediction:5 , Ans: 5
Postirior(log):

0:0.100254
1:0.166544
2:0.103950
3:0.090514
4:0.084599
5:0.086590
6:0.117543
7:0.081581
8:0.087262
9:0.081162
Prediction:9 , Ans: 7
Postirior(log):

0:0.092349
1:0.164405
2:0.086688
3:0.087070
4:0.090872
5:0.088918
6:0.088685
7:0.107962
8:0.088369
9:0.104680
Prediction:2 , Ans: 2
Postirior(log):

0:0.081155
1:0.220104
2:0.066441
3:0.087551
4:0.089842
5:0.080841
6:0.078064
7:0.109761
8:0.083625
9:0.102615
Prediction:2 , Ans: 2
Postiri

3:0.097974
4:0.096413
5:0.097640
6:0.107140
7:0.099684
8:0.096197
9:0.095829
Prediction:9 , Ans: 4
Postirior(log):

0:0.094741
1:0.128104
2:0.098045
3:0.096181
4:0.095142
5:0.095017
6:0.101822
7:0.103040
8:0.093528
9:0.094380
Prediction:8 , Ans: 0
Postirior(log):

0:0.102998
1:0.095463
2:0.097113
3:0.097869
4:0.098324
5:0.096790
6:0.093415
7:0.113576
8:0.098065
9:0.106386
Prediction:6 , Ans: 6
Postirior(log):

0:0.095767
1:0.174191
2:0.085568
3:0.079240
4:0.089634
5:0.087233
6:0.111762
7:0.089685
8:0.085859
9:0.101062
Prediction:3 , Ans: 3
Postirior(log):

0:0.091464
1:0.228197
2:0.088928
3:0.090528
4:0.074813
5:0.082793
6:0.099833
7:0.081377
8:0.087982
9:0.074085
Prediction:9 , Ans: 9
Postirior(log):

0:0.091316
1:0.119628
2:0.087854
3:0.087735
4:0.107034
5:0.087083
6:0.106327
7:0.112068
8:0.090538
9:0.110415
Prediction:5 , Ans: 5
Postirior(log):

0:0.088355
1:0.171270
2:0.067481
3:0.076517
4:0.083705
5:0.080476
6:0.091488
7:0.135658
8:0.081059
9:0.123991
Prediction:2 , Ans: 2
Postiri

Prediction:9 , Ans: 9
Postirior(log):

0:0.103545
1:0.146485
2:0.097169
3:0.095014
4:0.088921
5:0.093014
6:0.103751
7:0.092245
8:0.091721
9:0.088133
Prediction:9 , Ans: 4
Postirior(log):

0:0.109304
1:0.087220
2:0.101744
3:0.099583
4:0.100657
5:0.098975
6:0.105166
7:0.102090
8:0.097742
9:0.097519
Prediction:1 , Ans: 1
Postirior(log):

0:0.101792
1:0.163107
2:0.097240
3:0.093147
4:0.085372
5:0.088450
6:0.108523
7:0.088721
8:0.089094
9:0.084555
Prediction:9 , Ans: 5
Postirior(log):

0:0.093625
1:0.173919
2:0.075261
3:0.067333
4:0.091459
5:0.086117
6:0.124318
7:0.091184
8:0.084008
9:0.112777
Prediction:3 , Ans: 3
Postirior(log):

0:0.103170
1:0.140072
2:0.095587
3:0.091795
4:0.091295
5:0.091307
6:0.109252
7:0.097373
8:0.088371
9:0.091778
Prediction:8 , Ans: 8
Postirior(log):

0:0.080679
1:0.189728
2:0.087953
3:0.090732
4:0.089376
5:0.086715
6:0.090581
7:0.104151
8:0.085448
9:0.094639
Prediction:0 , Ans: 0
Postirior(log):

0:0.098644
1:0.100820
2:0.094492
3:0.093095
4:0.100610
5:0.096575
6

2:0.102648
3:0.097981
4:0.097178
5:0.098100
6:0.105554
7:0.094694
8:0.098375
9:0.096454
Prediction:7 , Ans: 7
Postirior(log):

0:0.086496
1:0.175007
2:0.076274
3:0.086767
4:0.092866
5:0.090409
6:0.069536
7:0.116423
8:0.094459
9:0.111763
Prediction:6 , Ans: 6
Postirior(log):

0:0.111347
1:0.099434
2:0.105011
3:0.096891
4:0.095252
5:0.095373
6:0.117329
7:0.093181
8:0.093861
9:0.092320
Prediction:9 , Ans: 8
Postirior(log):

0:0.105613
1:0.141387
2:0.101685
3:0.091177
4:0.088585
5:0.090319
6:0.117585
7:0.088608
8:0.088768
9:0.086274
Prediction:9 , Ans: 8
Postirior(log):

0:0.090588
1:0.192315
2:0.098724
3:0.094323
4:0.084283
5:0.088065
6:0.106348
7:0.078271
8:0.087339
9:0.079745
Prediction:7 , Ans: 7
Postirior(log):

0:0.106720
1:0.104926
2:0.104279
3:0.092112
4:0.093213
5:0.093268
6:0.129765
7:0.093725
8:0.091101
9:0.090891
Prediction:9 , Ans: 3
Postirior(log):

0:0.096467
1:0.223790
2:0.081908
3:0.079759
4:0.080475
5:0.093445
6:0.094193
7:0.076098
8:0.083446
9:0.090418
Prediction:7 , Ans

...

9:0.091983
Prediction:8 , Ans: 8
Postirior(log):

0:0.091893
1:0.120362
2:0.092473
3:0.089532
4:0.095985
5:0.088175
6:0.093791
7:0.128102
8:0.088164
9:0.111523
Prediction:8 , Ans: 5
Postirior(log):

0:0.086961
1:0.189040
2:0.077866
3:0.085133
4:0.089043
5:0.084155
6:0.070975
7:0.121219
8:0.088097
9:0.107511
Prediction:6 , Ans: 6
Postirior(log):

0:0.093678
1:0.186783
2:0.089383
3:0.090584
4:0.082283
5:0.086545
6:0.088092
7:0.102188
8:0.087140
9:0.093323
Prediction:4 , Ans: 4
Postirior(log):

0:0.105466
1:0.140679
2:0.110380
3:0.091034
4:0.089389
5:0.088732
6:0.130935
7:0.075135
8:0.090910
9:0.077339
Prediction:7 , Ans: 7
Postirior(log):

0:0.086381
1:0.210101
2:0.080172
3:0.086207
4:0.084715
5:0.083760
6:0.074084
7:0.110085
8:0.085707
9:0.098787
Prediction:6 , Ans: 6
Postirior(log):

0:0.080657
1:0.156884
2:0.074252
3:0.078329
4:0.097691
5:0.079757
6:0.091780
7:0.139275
8:0.079016
9:0.122359
Prediction:2 , Ans: 2
Postirior(log):

0:0.085490
1:0.142111
2:0.075482
3:0.081194
4:0.103266
5

8:0.081713
9:0.098442
Prediction:0 , Ans: 0
Postirior(log):

0:0.067568
1:0.234982
2:0.079282
3:0.099700
4:0.079060
5:0.084122
6:0.072800
7:0.098447
8:0.084668
9:0.099370
Prediction:0 , Ans: 0
Postirior(log):

0:0.106911
1:0.086392
2:0.096346
3:0.095183
4:0.101815
5:0.095098
6:0.112758
7:0.110226
8:0.092474
9:0.102797
Prediction:1 , Ans: 1
Postirior(log):

0:0.090705
1:0.155744
2:0.085625
3:0.091733
4:0.088726
5:0.083266
6:0.070117
7:0.129270
8:0.091544
9:0.113270
Prediction:6 , Ans: 6
Postirior(log):

0:0.088121
1:0.174303
2:0.083877
3:0.091382
4:0.086002
5:0.081880
6:0.071278
7:0.123397
8:0.089333
9:0.110429
Prediction:6 , Ans: 6
Postirior(log):

0:0.094886
1:0.125227
2:0.097406
3:0.099012
4:0.090070
5:0.085509
6:0.120117
7:0.103921
8:0.082536
9:0.101316
Prediction:8 , Ans: 8
Postirior(log):

0:0.083253
1:0.101455
2:0.080059
3:0.080112
4:0.100573
5:0.080607
6:0.106778
7:0.157489
8:0.079598
9:0.130075
Prediction:8 , Ans: 2
Postirior(log):

0:0.100049
1:0.150476
2:0.092936
3:0.093828
4

4:0.087450
5:0.076863
6:0.133416
7:0.090654
8:0.078705
9:0.094556
Prediction:5 , Ans: 5
Postirior(log):

0:0.103506
1:0.100945
2:0.100055
3:0.096302
4:0.097476
5:0.096169
6:0.107565
7:0.107670
8:0.094075
9:0.096238
Prediction:8 , Ans: 8
Postirior(log):

0:0.099425
1:0.101816
2:0.102182
3:0.095952
4:0.098294
5:0.095468
6:0.109962
7:0.104683
8:0.095380
9:0.096838
Prediction:8 , Ans: 5
Postirior(log):

0:0.095361
1:0.102300
2:0.092063
3:0.090028
4:0.099885
5:0.092360
6:0.096562
7:0.124282
8:0.092293
9:0.114867
Prediction:3 , Ans: 3
Postirior(log):

0:0.106132
1:0.127618
2:0.099381
3:0.096503
4:0.092011
5:0.094905
6:0.101278
7:0.096649
8:0.094086
9:0.091437
Prediction:9 , Ans: 4
Postirior(log):

0:0.102510
1:0.126177
2:0.087341
3:0.094660
4:0.096807
5:0.094206
6:0.081663
7:0.115443
8:0.095367
9:0.105826
Prediction:6 , Ans: 6
Postirior(log):

0:0.107644
1:0.088274
2:0.100942
3:0.099109
4:0.100545
5:0.099129
6:0.099114
7:0.105608
8:0.098868
9:0.100767
Prediction:1 , Ans: 1
Postirior(log):

0


0:0.087959
1:0.157070
2:0.094690
3:0.093120
4:0.092776
5:0.088531
6:0.106716
7:0.096083
8:0.087427
9:0.095628
Prediction:8 , Ans: 0
Postirior(log):

0:0.108670
1:0.086407
2:0.099089
3:0.097454
4:0.101578
5:0.097567
6:0.110786
7:0.104144
8:0.095568
9:0.098736
Prediction:1 , Ans: 1
Postirior(log):

0:0.090636
1:0.100311
2:0.082175
3:0.086337
4:0.104521
5:0.085338
6:0.104978
7:0.139225
8:0.083168
9:0.123311
Prediction:2 , Ans: 2
Postirior(log):

0:0.094347
1:0.133157
2:0.090806
3:0.089724
4:0.095335
5:0.086953
6:0.123847
7:0.101947
8:0.084425
9:0.099461
Prediction:8 , Ans: 3
Postirior(log):

0:0.099321
1:0.160990
2:0.095636
3:0.095360
4:0.087909
5:0.091593
6:0.096424
7:0.093069
8:0.091901
9:0.087797
Prediction:9 , Ans: 4
Postirior(log):

0:0.098312
1:0.109844
2:0.101487
3:0.101153
4:0.096315
5:0.089073
6:0.106298
7:0.103402
8:0.089793
9:0.104322
Prediction:5 , Ans: 5
Postirior(log):

0:0.088782
1:0.136255
2:0.086410
3:0.090594
4:0.093465
5:0.085526
6:0.083757
7:0.125537
8:0.092229
9:0.11

3:0.078457
4:0.097590
5:0.076290
6:0.107249
7:0.102276
8:0.076701
9:0.105435
Prediction:2 , Ans: 2
Postirior(log):

0:0.097699
1:0.085214
2:0.092258
3:0.095064
4:0.100469
5:0.091142
6:0.108614
7:0.122363
8:0.090691
9:0.116486
Prediction:1 , Ans: 1
Postirior(log):

0:0.106562
1:0.085521
2:0.096137
3:0.097315
4:0.100777
5:0.096627
6:0.108102
7:0.109366
8:0.094604
9:0.104988
Prediction:1 , Ans: 1
Postirior(log):

0:0.103434
1:0.123369
2:0.108387
3:0.096024
4:0.093254
5:0.093653
6:0.119334
7:0.083583
8:0.094447
9:0.084515
Prediction:7 , Ans: 7
Postirior(log):

0:0.110547
1:0.096983
2:0.108002
3:0.094909
4:0.094920
5:0.094669
6:0.124877
7:0.090696
8:0.093747
9:0.090649
Prediction:9 , Ans: 7
Postirior(log):

0:0.106394
1:0.097298
2:0.098614
3:0.097332
4:0.099037
5:0.097567
6:0.098957
7:0.107714
8:0.096016
9:0.101072
Prediction:8 , Ans: 8
Postirior(log):

0:0.103248
1:0.148455
2:0.100419
3:0.095358
4:0.087767
5:0.091819
6:0.106519
7:0.089018
8:0.091062
9:0.086335
Prediction:9 , Ans: 4
Postiri

In [4]:
#HW2-2
def online_learning(a,b):
    with open('test.txt','rb') as f:
        num=0
        for line in f.readlines():
            num+=1
            string=str(line.strip()).split('b')[1]
            print('case %d: %s'%(num,string))
            list_str=[1 if i=='1' else 0 for i in list(string)[1:-1]]
            p=np.sum(list_str)/len(list_str)
            print('Likelihood: %f'%binom.pmf(np.sum(list_str), len(list_str), p))
            print('Beta prior: a=%d b=%d'%(a,b))
            print('Beta posterior: a=%d b=%d'%(a+np.sum(list_str),b+len(list_str)-np.sum(list_str)))
            a=a+np.sum(list_str)
            b=b+len(list_str)-np.sum(list_str)

online_learning(0,0)

case 1: '0101010101001011010101'
Likelihood: 0.168188
Beta prior: a=0 b=0
Beta posterior: a=11 b=11
case 2: '0110101'
Likelihood: 0.293755
Beta prior: a=11 b=11
Beta posterior: a=15 b=14
case 3: '010110101101'
Likelihood: 0.228605
Beta prior: a=15 b=14
Beta posterior: a=22 b=19
case 4: '0101101011101011010'
Likelihood: 0.182869
Beta prior: a=22 b=19
Beta posterior: a=33 b=27
case 5: '111101100011110'
Likelihood: 0.214307
Beta prior: a=33 b=27
Beta posterior: a=43 b=32
case 6: '101110111000110'
Likelihood: 0.206598
Beta prior: a=43 b=32
Beta posterior: a=52 b=38
case 7: '1010010111'
Likelihood: 0.250823
Beta prior: a=52 b=38
Beta posterior: a=58 b=42
case 8: '11101110110'
Likelihood: 0.261968
Beta prior: a=58 b=42
Beta posterior: a=66 b=45
case 9: '01000111101'
Likelihood: 0.236091
Beta prior: a=66 b=45
Beta posterior: a=72 b=50
case 10: '110100111'
Likelihood: 0.273129
Beta prior: a=72 b=50
Beta posterior: a=78 b=53
case 11: '01101010111'
Likelihood: 0.243849
Beta prior: a=78 b=53
Beta